# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.48,89,98,8.30,TF,1730173595
1,1,grytviken,-54.2811,-36.5092,-0.31,96,100,2.37,GS,1730173597
2,2,yellowknife,62.4560,-114.3525,-3.71,86,100,5.14,CA,1730173598
3,3,puerto natales,-51.7236,-72.4875,8.42,82,82,5.39,CL,1730173599
4,4,waitangi,-43.9535,-176.5597,14.90,83,28,1.79,NZ,1730173600


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot (5)
#formatting so that the size of each data point correlates to the humidity in the associated city. (6)
map_plot_1 = city_data_df.hvplot.points("Lng", 
                                        "Lat", 
                                        geo=True, 
                                        tiles="OSM",
                                        xlim = (-180,180),
                                       size = "Humidity",
                                       color = "City",
                                       scale = 0.7)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values (7)
good_weather_df = city_data_df.loc[(city_data_df["Max Temp"]>= 21) & (city_data_df["Max Temp"]<= 27) & (city_data_df["Wind Speed"]<= 4.5) & (city_data_df["Cloudiness"]== 0)]

# Drop any rows with null values (8)
good_weather_df = good_weather_df.dropna(how='any')

# Display sample data
good_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,remire-montjoly,4.9167,-52.2667,23.02,100,0,0.00,GF,1730173619
55,55,icano,-28.9193,-65.3282,21.36,46,0,2.02,AR,1730173667
114,114,lazaro cardenas,17.9583,-102.2000,25.94,83,0,1.71,MX,1730173737
218,218,heze,35.2431,115.4411,21.81,43,0,4.18,CN,1730173862
229,229,irymple,-34.2500,142.1667,26.16,27,0,0.89,AU,1730173875


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity (9)
hotel_df = good_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API (10)
hotel_df["Hotel Name"] = ""

#reset index, for improved iteration in following steps (11)
hotel_df = hotel_df.reset_index(drop=True)

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,18,remire-montjoly,4.9167,-52.2667,23.02,100,0,0.00,GF,1730173619,
1,55,icano,-28.9193,-65.3282,21.36,46,0,2.02,AR,1730173667,
2,114,lazaro cardenas,17.9583,-102.2000,25.94,83,0,1.71,MX,1730173737,
3,218,heze,35.2431,115.4411,21.81,43,0,4.18,CN,1730173862,
4,229,irymple,-34.2500,142.1667,26.16,27,0,0.89,AU,1730173875,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
# sources for this code block (12) (13)
radius = 10000
params = {"categories":"accommodation.hotel",
         "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.iloc[index,2]
    lon = hotel_df.iloc[index,3]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
icano - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
heze - nearest hotel: No hotel found
irymple - nearest hotel: Chaffey Motor Inn
xiaoweizhai - nearest hotel: 7Days Inn Duyun Bus Terminal Station
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
gebeit - nearest hotel: No hotel found
la mira - nearest hotel: Hotel La Loma
broken hill - nearest hotel: Ibis Styles
duyun - nearest hotel: 7Days Inn Duyun Bus Terminal Station
zhangjiajie - nearest hotel: 湘云韵国际大酒店
haridwar - nearest hotel: Pilibhit House, Haridwar - IHCL SeleQtions
maracacume - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
truth or consequences - nearest hotel: Travelodge


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,18,remire-montjoly,4.9167,-52.2667,23.02,100,0,0.00,GF,1730173619,Complexe Belova
1,55,icano,-28.9193,-65.3282,21.36,46,0,2.02,AR,1730173667,No hotel found
2,114,lazaro cardenas,17.9583,-102.2000,25.94,83,0,1.71,MX,1730173737,Hotel Sol del Pacífico
3,218,heze,35.2431,115.4411,21.81,43,0,4.18,CN,1730173862,No hotel found
4,229,irymple,-34.2500,142.1667,26.16,27,0,0.89,AU,1730173875,Chaffey Motor Inn
5,239,xiaoweizhai,26.1903,107.5125,21.60,52,0,1.85,CN,1730173886,7Days Inn Duyun Bus Terminal Station
6,246,ciudad lazaro cardenas,17.9583,-102.2000,25.94,83,0,1.71,MX,1730173894,Hotel Sol del Pacífico
7,268,gebeit,21.0667,36.3167,21.79,47,0,2.33,SD,1730173919,No hotel found
8,299,la mira,18.0350,-102.3266,24.61,87,0,1.86,MX,1730173956,Hotel La Loma
9,303,broken hill,-31.9500,141.4333,25.69,19,0,4.44,AU,1730173961,Ibis Styles


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [50]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo=True,
                                    tiles="OSM",
                                    xlim=(-180,180),
                                    ylim=(-70,70),
                                    size="Humidity",
                                    color="City",
                                    scale=2,
                                    hover_cols=["Hotel Name", "Country"] #(14)
                                   )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)